# 제품 이상여부 판별 프로젝트


## 1. 데이터 불러오기


### 필수 라이브러리


In [615]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### 데이터 읽어오기


In [616]:
RANDOM_STATE = 110

train_data = pd.read_csv("../../data/trim_train_data.csv")
test_data = pd.read_csv("../../data/trim_test_data.csv")

---

반복적으로 쓰는 툴 함수화

In [617]:
def plot_box(dataframe, column_name):
    """
    주어진 데이터프레임과 열 이름에 대해 박스 플롯을 그리는 함수.

    Parameters:
    dataframe (pd.DataFrame): 데이터프레임
    column_name (str): 열 이름
    """
    plt.figure(figsize=(10, 6))
    plt.boxplot(dataframe[column_name], vert=False)
    plt.xlabel(column_name)
    plt.title(f'Box Plot of {column_name}')
    plt.show()

In [618]:
import pandas as pd

def value_counts_ratio_count(df, col_name, target_name):
    """
    주어진 데이터프레임의 특정 열에 대해 각 값마다 타겟 변수의 비율과 갯수, 총 갯수를 출력하는 함수.

    Parameters:
    df (pd.DataFrame): 데이터프레임
    col_name (str): 열 이름
    target_name (str): 타겟 변수 이름
    """
    # 각 값마다 타겟 변수의 비율 계산
    value_counts = df.groupby(col_name)[target_name].value_counts(normalize=True).unstack().fillna(0)
    
    # 각 값마다 타겟 변수의 갯수 계산
    counts = df.groupby(col_name)[target_name].value_counts().unstack().fillna(0)
    
    # 각 값마다 총 갯수 계산
    total_counts = df[col_name].value_counts().rename('Total_Count')
    
    # 비율과 갯수를 합침
    result = value_counts.join(counts, lsuffix='_ratio', rsuffix='_count')
    
    # 총 갯수를 합침
    result = result.join(total_counts, on=col_name)
    
    # 출력 형식 조정
    result.index.name = 'variable'
    print(f"\n{col_name}별 {target_name} 비율 및 갯수\n")
    print(result.rename(columns=lambda x: x.split('_')[0]))

In [619]:
import pandas as pd

def summarize_grouped_data(df, group_by_columns):
    # 데이터프레임을 그룹화
    grouped_df = df.groupby(group_by_columns)
    
    # 결과를 저장할 리스트 초기화
    results = []
    
    # 그룹화된 데이터프레임의 내용을 확인하는 코드
    for name, group in grouped_df:
        # 그룹의 갯수 계산
        group_count = group.shape[0]
        
        # 'target' 변수의 'AdNormal' 비율과 갯수 계산
        adnormal_count = group['target'].value_counts().get('AbNormal', 0)
        adnormal_ratio = adnormal_count / group_count
        
        # 결과 리스트에 추가
        results.append([name, adnormal_count, adnormal_ratio, group_count])
    
    # 결과 리스트를 데이터프레임으로 변환
    results_df = pd.DataFrame(results, columns=['group', "'AdNormal' count", 'ratio', 'Total'])
    
    # 그룹화된 변수들의 이름을 제목행으로 출력
    print(f"Grouped by: {', '.join(group_by_columns)}")
    print()
    # 데이터프레임 출력
    print(results_df)

In [620]:
import pandas as pd
import matplotlib.pyplot as plt

def plot_abnormal_ratio(df, group_by_column, target_column, abnormal_value):
    # 데이터프레임을 그룹화
    grouped_df = df.groupby(group_by_column)
    
    # 결과를 저장할 리스트 초기화
    results = []
    
    # 그룹화된 데이터프레임의 내용을 확인하는 코드
    for name, group in grouped_df:
        # 그룹의 갯수 계산
        group_count = group.shape[0]
        
        # 'target' 변수의 'AbNormal' 비율과 갯수 계산
        abnormal_count = group[target_column].value_counts().get(abnormal_value, 0)
        abnormal_ratio = abnormal_count / group_count
        
        # 결과 리스트에 추가
        results.append([name, abnormal_count, abnormal_ratio, group_count])
    
    # 결과 리스트를 데이터프레임으로 변환
    results_df = pd.DataFrame(results, columns=['group', f"'{abnormal_value}' count", 'ratio', 'Total'])
    
    # 그래프 크기 설정
    plt.figure(figsize=(12, 6))
    
    # 막대 그래프 생성
    ax = results_df.plot(kind='bar', x='group', y='ratio', legend=False)
    
    # 각 막대 위에 AbNormal 갯수와 총 갯수 표시
    for i, (abnormal_count, total) in enumerate(zip(results_df[f"'{abnormal_value}' count"], results_df['Total'])):
        ax.text(i, results_df['ratio'][i], f'{abnormal_count} ({total})', ha='center', va='bottom', fontsize=8)
    
     # 그래프 제목 및 축 레이블 설정
    ax.set_title(f'{abnormal_value} Ratio by {group_by_column}')
    ax.set_xlabel(group_by_column)
    ax.set_ylabel(f'{abnormal_value} Ratio')
   
    # 그래프 출력
    plt.show()

---

## AutoClave

In [621]:
# 오타인 변수명 변경
train_data = train_data.rename(columns={'1st Pressure 1st Pressure Unit Time_AutoClave': '1st Pressure Unit Time_AutoClave'})
test_data = test_data.rename(columns={'1st Pressure 1st Pressure Unit Time_AutoClave': '1st Pressure Unit Time_AutoClave'})

In [622]:
# '_AutoClave'를 포함하는 열 이름 필터링
Process_Desc_col = train_data.filter(like='_AutoClave').columns

# 필터링된 열 이름 출력
print("필터링된 열 이름:")
for col in Process_Desc_col:
    print(col)

필터링된 열 이름:
Wip Line_AutoClave
Process Desc._AutoClave
Equipment_AutoClave
Model.Suffix_AutoClave
Workorder_AutoClave
Insp. Seq No._AutoClave
Insp Judge Code_AutoClave
1st Pressure Collect Result_AutoClave
1st Pressure Unit Time_AutoClave
1st Pressure Judge Value_AutoClave
2nd Pressure Collect Result_AutoClave
2nd Pressure Unit Time_AutoClave
2nd Pressure Judge Value_AutoClave
3rd Pressure Collect Result_AutoClave
3rd Pressure Unit Time_AutoClave
3rd Pressure Judge Value_AutoClave
Chamber Temp. Collect Result_AutoClave
Chamber Temp. Unit Time_AutoClave
Chamber Temp. Judge Value_AutoClave
GMES_ORIGIN_INSP_JUDGE_CODE Collect Result_AutoClave
GMES_ORIGIN_INSP_JUDGE_CODE Unit Time_AutoClave
GMES_ORIGIN_INSP_JUDGE_CODE Judge Value_AutoClave


fill 공정 관련 변수중 모든값이 결측값인 경우 drop

In [623]:
# target 열을 임시로 분리
target_train = train_data['target']
target_test = test_data['target']

# '_AutoClave'이 포함된 열만 필터링
fill_columns_train = [col for col in train_data.columns if '_AutoClave' in col]
fill_columns_test = [col for col in test_data.columns if '_AutoClave' in col]

# 모든 값이 NaN인 '_fill' 열 제거
train_data = train_data.drop(columns=[col for col in fill_columns_train if train_data[col].isna().all()])
test_data = test_data.drop(columns=[col for col in fill_columns_test if test_data[col].isna().all()])

# target 열을 다시 결합
train_data['target'] = target_train
test_data['target'] = target_test

In [624]:
# '_AutoClave'를 포함하는 열 이름 필터링
Process_Desc_col = train_data.filter(like='_AutoClave').columns

# 필터링된 열 이름 출력
print("필터링된 열 이름:")
for col in Process_Desc_col:
    print(col)

필터링된 열 이름:
Wip Line_AutoClave
Process Desc._AutoClave
Equipment_AutoClave
Model.Suffix_AutoClave
Workorder_AutoClave
Insp. Seq No._AutoClave
Insp Judge Code_AutoClave
1st Pressure Collect Result_AutoClave
1st Pressure Unit Time_AutoClave
1st Pressure Judge Value_AutoClave
2nd Pressure Collect Result_AutoClave
2nd Pressure Unit Time_AutoClave
2nd Pressure Judge Value_AutoClave
3rd Pressure Collect Result_AutoClave
3rd Pressure Unit Time_AutoClave
3rd Pressure Judge Value_AutoClave
Chamber Temp. Collect Result_AutoClave
Chamber Temp. Unit Time_AutoClave
Chamber Temp. Judge Value_AutoClave
GMES_ORIGIN_INSP_JUDGE_CODE Collect Result_AutoClave
GMES_ORIGIN_INSP_JUDGE_CODE Judge Value_AutoClave


In [625]:
value_counts_ratio_count(train_data, 'Wip Line_AutoClave', 'target')


Wip Line_AutoClave별 target 비율 및 갯수

          AbNormal    Normal  AbNormal  Normal  Total
variable                                             
IVI-OB6   0.058016  0.941984      2350   38156  40506


In [626]:
value_counts_ratio_count(train_data, 'Process Desc._AutoClave', 'target')


Process Desc._AutoClave별 target 비율 및 갯수

                AbNormal    Normal  AbNormal  Normal  Total
variable                                                   
Auto Clave Out  0.058016  0.941984      2350   38156  40506


In [627]:
value_counts_ratio_count(train_data, 'Equipment_AutoClave', 'target')


Equipment_AutoClave별 target 비율 및 갯수

                AbNormal    Normal  AbNormal  Normal  Total
variable                                                   
Auto Clave Out  0.058016  0.941984      2350   38156  40506


단일 고윳값 제거

In [628]:
# 삭제할 열 이름 정의
columns_to_drop = [
    'Wip Line_AutoClave'
    , 'Process Desc._AutoClave'
    , 'Equipment_AutoClave'
]

# train_data에서 열 삭제
train_data = train_data.drop(columns=columns_to_drop)

# test_data에서 열 삭제
test_data = test_data.drop(columns=columns_to_drop)

In [629]:
# '_AutoClave'를 포함하는 열 이름 필터링
Process_Desc_col = train_data.filter(like='_AutoClave').columns

# 필터링된 열 이름 출력
print("필터링된 열 이름:")
for col in Process_Desc_col:
    print(col)

필터링된 열 이름:
Model.Suffix_AutoClave
Workorder_AutoClave
Insp. Seq No._AutoClave
Insp Judge Code_AutoClave
1st Pressure Collect Result_AutoClave
1st Pressure Unit Time_AutoClave
1st Pressure Judge Value_AutoClave
2nd Pressure Collect Result_AutoClave
2nd Pressure Unit Time_AutoClave
2nd Pressure Judge Value_AutoClave
3rd Pressure Collect Result_AutoClave
3rd Pressure Unit Time_AutoClave
3rd Pressure Judge Value_AutoClave
Chamber Temp. Collect Result_AutoClave
Chamber Temp. Unit Time_AutoClave
Chamber Temp. Judge Value_AutoClave
GMES_ORIGIN_INSP_JUDGE_CODE Collect Result_AutoClave
GMES_ORIGIN_INSP_JUDGE_CODE Judge Value_AutoClave


In [630]:
value_counts_ratio_count(train_data, 'Model.Suffix_AutoClave', 'target')


Model.Suffix_AutoClave별 target 비율 및 갯수

             AbNormal    Normal  AbNormal  Normal  Total
variable                                                
AJX75334501  0.056712  0.943288      1918   31902  33820
AJX75334502  0.067847  0.932153       230    3160   3390
AJX75334503  0.271605  0.728395        44     118    162
AJX75334505  0.049336  0.950664       130    2505   2635
AJX75334506  0.062016  0.937984         8     121    129
AJX75334507  0.058065  0.941935        18     292    310
AJX75334508  0.033333  0.966667         2      58     60


In [631]:
value_counts_ratio_count(train_data, 'Workorder_AutoClave', 'target')


Workorder_AutoClave별 target 비율 및 갯수

            AbNormal    Normal  AbNormal  Normal  Total
variable                                               
3F1X5847-2  0.113208  0.886792      12.0    94.0    106
3F1X9643-1  0.047619  0.952381       7.0   140.0    147
3F1X9644-1  0.101911  0.898089      16.0   141.0    157
3F1X9648-1  0.062500  0.937500      10.0   150.0    160
3F1X9648-2  0.055556  0.944444       1.0    17.0     18
...              ...       ...       ...     ...    ...
4F1XB043-1  0.037037  0.962963       1.0    26.0     27
4F1XB596-1  0.020833  0.979167       3.0   141.0    144
4F1XB738-1  0.000000  1.000000       0.0    18.0     18
4F1XB758-1  0.025974  0.974026       2.0    75.0     77
4F1XB758-2  0.222222  0.777778      10.0    35.0     45

[663 rows x 5 columns]


In [632]:
import pandas as pd

# 'Workorder_AutoClave' 열에서 '-' 다음 숫자 값 추출 및 '000' 제거
train_data['Workorder_AutoClave'] = train_data['Workorder_AutoClave'].str.replace(r'-(\d+)', lambda x: '-' + x.group(1).lstrip('0'), regex=True)

xxxxx-0001 -> xxxxx-1 변환 코드

In [633]:
value_counts_ratio_count(train_data, 'Insp. Seq No._AutoClave', 'target')


Insp. Seq No._AutoClave별 target 비율 및 갯수

          AbNormal    Normal  AbNormal  Normal  Total
variable                                             
1         0.058016  0.941984      2350   38156  40506


In [634]:
value_counts_ratio_count(train_data, 'Insp Judge Code_AutoClave', 'target')


Insp Judge Code_AutoClave별 target 비율 및 갯수

          AbNormal    Normal  AbNormal  Normal  Total
variable                                             
OK        0.058016  0.941984      2350   38156  40506


In [635]:
import pandas as pd

# # pandas 출력 옵션 설정
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth', None)

# # 데이터 요약
summary_df = summarize_grouped_data(train_data, [
    'Insp. Seq No._AutoClave'
    , 'Insp Judge Code_AutoClave'
    ])

Grouped by: Insp. Seq No._AutoClave, Insp Judge Code_AutoClave

     group  'AdNormal' count     ratio  Total
0  (1, OK)              2350  0.058016  40506


Insp. Seq No._AutoClave, Insp Judge Code_AutoClave 둘다 단일 고윳값  
-> drop

In [636]:
# 삭제할 열 이름 정의
columns_to_drop = [
    'Insp. Seq No._AutoClave'
    , 'Insp Judge Code_AutoClave'
    ]

# train_data에서 열 삭제
train_data = train_data.drop(columns=columns_to_drop)

# test_data에서 열 삭제
test_data = test_data.drop(columns=columns_to_drop)

In [637]:
# '_AutoClave'를 포함하는 열 이름 필터링
Process_Desc_col = train_data.filter(like='_AutoClave').columns

# 필터링된 열 이름 출력
print("필터링된 열 이름:")
for col in Process_Desc_col:
    print(col)

필터링된 열 이름:
Model.Suffix_AutoClave
Workorder_AutoClave
1st Pressure Collect Result_AutoClave
1st Pressure Unit Time_AutoClave
1st Pressure Judge Value_AutoClave
2nd Pressure Collect Result_AutoClave
2nd Pressure Unit Time_AutoClave
2nd Pressure Judge Value_AutoClave
3rd Pressure Collect Result_AutoClave
3rd Pressure Unit Time_AutoClave
3rd Pressure Judge Value_AutoClave
Chamber Temp. Collect Result_AutoClave
Chamber Temp. Unit Time_AutoClave
Chamber Temp. Judge Value_AutoClave
GMES_ORIGIN_INSP_JUDGE_CODE Collect Result_AutoClave
GMES_ORIGIN_INSP_JUDGE_CODE Judge Value_AutoClave


분석에 앞서 OK 값을 가지는 변수들에 대한 처리를 먼저 진행

In [645]:
summarize_grouped_data(train_data, [
    '1st Pressure Judge Value_AutoClave'
    , '2nd Pressure Judge Value_AutoClave'
    , '3rd Pressure Judge Value_AutoClave'])

Grouped by: 1st Pressure Judge Value_AutoClave, 2nd Pressure Judge Value_AutoClave, 3rd Pressure Judge Value_AutoClave

          group  'AdNormal' count     ratio  Total
0  (OK, OK, OK)              2350  0.058016  40506


모든 값 OK 값 -> drop

In [646]:
# 삭제하려는 열 목록
columns_to_drop = [
    '1st Pressure Judge Value_AutoClave'
    , '2nd Pressure Judge Value_AutoClave'
    , '3rd Pressure Judge Value_AutoClave']

# train_data에서 열 삭제
train_data = train_data.drop(columns=[col for col in columns_to_drop if col in train_data.columns])

# test_data에서 열 삭제
test_data = test_data.drop(columns=[col for col in columns_to_drop if col in test_data.columns])

4개의 변수 모두 동일 행에 대해서 ok 값을 가짐

In [647]:
summarize_grouped_data(train_data, [
    'GMES_ORIGIN_INSP_JUDGE_CODE Collect Result_AutoClave'
    ,'GMES_ORIGIN_INSP_JUDGE_CODE Judge Value_AutoClave'])

Grouped by: GMES_ORIGIN_INSP_JUDGE_CODE Collect Result_AutoClave, GMES_ORIGIN_INSP_JUDGE_CODE Judge Value_AutoClave

      group  'AdNormal' count     ratio  Total
0  (OK, OK)               496  0.043921  11293


위의 Ok 값은 초기 밀려진 데이터에 대해서 생성된 값으로 판단  
-> drop

In [648]:
# 삭제하려는 열 목록
columns_to_drop = [
    'GMES_ORIGIN_INSP_JUDGE_CODE Collect Result_AutoClave'
    , 'GMES_ORIGIN_INSP_JUDGE_CODE Judge Value_AutoClave']

# train_data에서 열 삭제
train_data = train_data.drop(columns=[col for col in columns_to_drop if col in train_data.columns])

# test_data에서 열 삭제
test_data = test_data.drop(columns=[col for col in columns_to_drop if col in test_data.columns])

In [649]:
train_data['Chamber Temp. Judge Value_AutoClave'].value_counts()

OK    29112
NG    11394
Name: Chamber Temp. Judge Value_AutoClave, dtype: int64

Chamber Temp. Judge Value_AutoClave 변수명 Chamber_Temp_OKNG_AutoClave으로 변경  
-> 변수명 간결화 목적

In [652]:
# 'Chamber_Temp_OKNG_AutoClave' 변수 추가
train_data['Chamber_Temp_OKNG_AutoClave'] = train_data['Chamber Temp. Judge Value_AutoClave']
test_data['Chamber_Temp_OKNG_AutoClave'] = test_data['Chamber Temp. Judge Value_AutoClave']

# 'Chamber Temp. Judge Value_AutoClave' 열 삭제
train_data.drop(columns=['Chamber Temp. Judge Value_AutoClave'], inplace=True)
test_data.drop(columns=['Chamber Temp. Judge Value_AutoClave'], inplace=True)

In [654]:
# '_AutoClave'를 포함하는 열 이름 필터링
Process_Desc_col = train_data.filter(like='_AutoClave').columns

# 필터링된 열 이름 출력
print("필터링된 열 이름:")
for col in Process_Desc_col:
    print(col)

필터링된 열 이름:
Model.Suffix_AutoClave
Workorder_AutoClave
1st Pressure Collect Result_AutoClave
1st Pressure Unit Time_AutoClave
2nd Pressure Collect Result_AutoClave
2nd Pressure Unit Time_AutoClave
3rd Pressure Collect Result_AutoClave
3rd Pressure Unit Time_AutoClave
Chamber Temp. Collect Result_AutoClave
Chamber Temp. Unit Time_AutoClave
Chamber_Temp_OKNG_AutoClave
